In [1]:
%pip install visualkeras

Note: you may need to restart the kernel to use updated packages.


In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
import numpy as np

2023-11-21 22:50:53.054904: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-21 22:50:54.046632: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import os
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, BatchNormalization 
from keras.preprocessing import image
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [3]:
data_path = "data_citrus_level/"
CATEGORIES = ["black_spot", "black_spot_early", "canker", "canker_early", "greening", "greening_early", "melanose", "melanose_early", "healthy"]
IMG_SIZE = 224

#create training data
training = []
def createTrainingData():
    for category in CATEGORIES:
        path = os.path.join(data_path, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            img_array = cv2.imread(os.path.join(path,img))
            new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
            training.append([new_array, class_num])

createTrainingData()

In [4]:
import random
random.shuffle(training)

In [4]:
X = []
y = []
for features, label in training:
    X.append(features)
    y.append(label)

In [5]:
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)

#Normalize X
X = X.astype('float32')
X /= 255

Y = keras.utils.to_categorical(y)
print(X.shape)
print(Y.shape)   

(1152, 224, 224, 3)
(1152, 9)


In [ ]:
print(Y[10])

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.10)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.17)

In [7]:
#CNN1
cnn1 = Sequential()
cnn1.add(Conv2D(32, kernel_size = (3,3), padding = "same", activation="relu", input_shape = (224, 224, 3)))
cnn1.add(MaxPooling2D(pool_size=(2, 2)))
cnn1.add(BatchNormalization())
cnn1.add(Conv2D(32, kernel_size = (3,3), padding = "same", activation="relu"))
cnn1.add(MaxPooling2D(pool_size=(2, 2)))
cnn1.add(Dropout(0.5))
cnn1.add(Flatten())
cnn1.add(Dense(256, activation = 'relu'))
cnn1.add(Dense(9, activation='softmax'))
cnn1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 32)      0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 112, 112, 32)      128       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 56, 56, 32)        0         
 g2D)                                                            
                                                        

2023-11-21 22:51:25.381249: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8727 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:03:00.0, compute capability: 7.5


In [8]:
cnn1.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [11]:
hist1 = cnn1.fit(X_train, y_train, batch_size = 16, epochs = 30, validation_data = (X_val, y_val))

Epoch 1/30
54/54 [==============================] - 2s 31ms/step - loss: 0.2401 - accuracy: 0.9267 - val_loss: 4.7491 - val_accuracy: 0.4350
Epoch 2/30
54/54 [==============================] - 1s 24ms/step - loss: 0.1919 - accuracy: 0.9523 - val_loss: 4.2434 - val_accuracy: 0.3785
Epoch 3/30
54/54 [==============================] - 1s 23ms/step - loss: 0.1277 - accuracy: 0.9569 - val_loss: 6.0237 - val_accuracy: 0.3164
Epoch 4/30
54/54 [==============================] - 1s 23ms/step - loss: 0.1492 - accuracy: 0.9604 - val_loss: 7.1929 - val_accuracy: 0.3842
Epoch 5/30
54/54 [==============================] - 1s 23ms/step - loss: 0.0376 - accuracy: 0.9872 - val_loss: 11.8908 - val_accuracy: 0.2090
Epoch 6/30
54/54 [==============================] - 1s 24ms/step - loss: 0.0221 - accuracy: 0.9953 - val_loss: 3.3187 - val_accuracy: 0.5819
Epoch 7/30
54/54 [==============================] - 1s 23ms/step - loss: 0.0305 - accuracy: 0.9930 - val_loss: 2.3944 - val_accuracy: 0.5932
Epoch 8/30
5

In [12]:
cnn1.evaluate(X_test, y_test)

4/4 [==============================] - 0s 11ms/step - loss: 3.6311 - accuracy: 0.5431


[3.631108522415161, 0.5431034564971924]

In [16]:
print(y_test)

[[0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [12]:
y_pred = cnn1.predict(X_test)
print(y_pred)

39/39 [==============================] - 1s 9ms/step
[[3.68895770e-22 2.12484405e-11 1.00000000e+00 ... 3.34794855e-14
  2.91822405e-17 8.21483396e-23]
 [3.10468119e-18 3.03482970e-20 4.90016021e-07 ... 2.74658396e-22
  2.20696649e-23 5.20402973e-05]
 [0.00000000e+00 1.18917828e-32 1.53337919e-32 ... 0.00000000e+00
  0.00000000e+00 3.47671346e-20]
 ...
 [8.88502242e-22 8.12023297e-30 5.60900826e-10 ... 1.06863338e-25
  4.03789915e-36 2.45676489e-07]
 [6.02985321e-37 9.04335195e-38 2.59763351e-31 ... 0.00000000e+00
  0.00000000e+00 1.10371115e-23]
 [3.16480668e-15 5.62290192e-24 1.10442082e-08 ... 6.26689039e-17
  8.62287637e-23 7.25516327e-13]]


In [20]:
y_pred = (y_pred >= 0.9).astype("int32")
print(y_pred)

[[0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [13]:
cnn1.save('cnn1.h5')

/home/tower1/.local/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [14]:
#CNN2
cnn2 = Sequential()
cnn2.add(Conv2D(32, kernel_size = (3,3), padding = "same", activation="relu", input_shape = (224, 224, 3)))
cnn2.add(MaxPooling2D(pool_size=(2, 2)))
cnn2.add(BatchNormalization())
cnn2.add(Conv2D(32, kernel_size = (3,3), padding = "same", activation="relu"))
cnn2.add(MaxPooling2D(pool_size=(2, 2)))
cnn2.add(Dropout(0.5))
cnn2.add(Flatten())
cnn2.add(Dense(512,activation = 'relu'))
cnn2.add(Dense(256, activation = 'relu'))
cnn2.add(Dense(9, activation='softmax'))
cnn2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 112, 112, 32)      0         
 g2D)                                                            
                                                                 
 batch_normalization_1 (Bat  (None, 112, 112, 32)      128       
 chNormalization)                                                
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 32)      9248      
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 56, 56, 32)        0         
 g2D)                                                            
                                                      

In [15]:
cnn2.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [18]:
hist2 = cnn2.fit(X_train, y_train, batch_size= 16, epochs = 30, validation_data = (X_val, y_val))

Epoch 1/30
54/54 [==============================] - 2s 31ms/step - loss: 3.7235e-04 - accuracy: 1.0000 - val_loss: 1.7278 - val_accuracy: 0.6610
Epoch 2/30
54/54 [==============================] - 1s 25ms/step - loss: 2.9362e-04 - accuracy: 1.0000 - val_loss: 1.7405 - val_accuracy: 0.6610
Epoch 3/30
54/54 [==============================] - 1s 26ms/step - loss: 3.1657e-04 - accuracy: 1.0000 - val_loss: 1.7521 - val_accuracy: 0.6780
Epoch 4/30
54/54 [==============================] - 1s 25ms/step - loss: 2.5103e-04 - accuracy: 1.0000 - val_loss: 1.7780 - val_accuracy: 0.6780
Epoch 5/30
54/54 [==============================] - 1s 25ms/step - loss: 2.1627e-04 - accuracy: 1.0000 - val_loss: 1.7738 - val_accuracy: 0.6780
Epoch 6/30
54/54 [==============================] - 1s 25ms/step - loss: 2.1541e-04 - accuracy: 1.0000 - val_loss: 1.7857 - val_accuracy: 0.6949
Epoch 7/30
54/54 [==============================] - 1s 25ms/step - loss: 1.6085e-04 - accuracy: 1.0000 - val_loss: 1.8176 - val_ac

In [19]:
cnn2.evaluate(X_test, y_test)

4/4 [==============================] - 0s 11ms/step - loss: 2.2920 - accuracy: 0.6379


[2.2919564247131348, 0.6379310488700867]

In [20]:
cnn2.save('cnn2.h5')

In [21]:
#CNN3
cnn3 = Sequential()
cnn3.add(Conv2D(32, kernel_size = (3,3), padding = "same", activation="relu", input_shape = (224, 224, 3)))
cnn3.add(MaxPooling2D(pool_size=(2, 2)))
cnn3.add(BatchNormalization())
cnn3.add(Conv2D(32, kernel_size = (3,3), padding = "same", activation="relu"))
cnn3.add(MaxPooling2D(pool_size=(2, 2)))
cnn3.add(Dropout(0.5))
cnn3.add(Flatten())
cnn3.add(Dense(128, activation = 'relu'))
cnn3.add(Dense(64, activation="relu"))
cnn3.add(Dense(32, activation="relu"))
cnn3.add(Dense(9, activation='softmax'))
cnn3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 112, 112, 32)      0         
 g2D)                                                            
                                                                 
 batch_normalization_2 (Bat  (None, 112, 112, 32)      128       
 chNormalization)                                                
                                                                 
 conv2d_5 (Conv2D)           (None, 112, 112, 32)      9248      
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 56, 56, 32)        0         
 g2D)                                                            
                                                      

In [22]:
cnn3.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [23]:
hist3 = cnn3.fit(X_train, y_train, batch_size= 16, epochs = 30, validation_data = (X_val, y_val))

Epoch 1/30


2023-11-21 22:57:47.720118: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_2/dropout_2/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


54/54 [==============================] - 5s 30ms/step - loss: 2.8930 - accuracy: 0.2678 - val_loss: 2.1271 - val_accuracy: 0.2316
Epoch 2/30
54/54 [==============================] - 1s 23ms/step - loss: 1.2699 - accuracy: 0.5390 - val_loss: 1.9813 - val_accuracy: 0.2542
Epoch 3/30
54/54 [==============================] - 1s 23ms/step - loss: 0.8501 - accuracy: 0.6962 - val_loss: 1.9020 - val_accuracy: 0.3277
Epoch 4/30
54/54 [==============================] - 1s 23ms/step - loss: 0.5289 - accuracy: 0.8033 - val_loss: 1.7289 - val_accuracy: 0.4124
Epoch 5/30
54/54 [==============================] - 1s 23ms/step - loss: 0.4580 - accuracy: 0.8440 - val_loss: 1.5913 - val_accuracy: 0.4011
Epoch 6/30
54/54 [==============================] - 1s 23ms/step - loss: 0.2317 - accuracy: 0.9255 - val_loss: 1.3099 - val_accuracy: 0.5763
Epoch 7/30
54/54 [==============================] - 1s 23ms/step - loss: 0.1369 - accuracy: 0.9604 - val_loss: 1.0879 - val_accuracy: 0.6158
Epoch 8/30
54/54 [======

In [24]:
cnn3.evaluate(X_test, y_test)

4/4 [==============================] - 0s 11ms/step - loss: 2.7042 - accuracy: 0.6207


[2.7041728496551514, 0.6206896305084229]

In [25]:
cnn3.save('cnn3.h5')

In [26]:
#CNN4
cnn4 = Sequential()
cnn4.add(Conv2D(32, kernel_size = (3,3), padding = "same", activation="relu", input_shape = (224, 224, 3)))
cnn4.add(MaxPooling2D(pool_size=(2, 2)))
cnn4.add(BatchNormalization())
cnn4.add(Conv2D(32, kernel_size = (3,3), padding = "same", activation="relu"))
cnn4.add(MaxPooling2D(pool_size=(2, 2)))
cnn4.add(Dropout(0.5))
cnn4.add(Flatten())
cnn4.add(Dense(128, activation = 'relu'))
cnn4.add(Dense(64, activation="relu"))
cnn4.add(Dense(9, activation='softmax'))
cnn4.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 112, 112, 32)      0         
 g2D)                                                            
                                                                 
 batch_normalization_3 (Bat  (None, 112, 112, 32)      128       
 chNormalization)                                                
                                                                 
 conv2d_7 (Conv2D)           (None, 112, 112, 32)      9248      
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 56, 56, 32)        0         
 g2D)                                                            
                                                      

In [27]:
cnn4.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [28]:
hist4 = cnn4.fit(X_train, y_train, batch_size= 16, epochs = 30, validation_data = (X_val, y_val))

Epoch 1/30


2023-11-21 23:00:31.021119: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_3/dropout_3/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


54/54 [==============================] - 4s 29ms/step - loss: 3.9460 - accuracy: 0.3260 - val_loss: 2.1006 - val_accuracy: 0.1864
Epoch 2/30
54/54 [==============================] - 1s 22ms/step - loss: 1.1490 - accuracy: 0.5704 - val_loss: 2.1582 - val_accuracy: 0.1186
Epoch 3/30
54/54 [==============================] - 1s 22ms/step - loss: 0.7074 - accuracy: 0.7439 - val_loss: 2.0894 - val_accuracy: 0.1695
Epoch 4/30
54/54 [==============================] - 1s 22ms/step - loss: 0.4323 - accuracy: 0.8533 - val_loss: 1.7788 - val_accuracy: 0.3785
Epoch 5/30
54/54 [==============================] - 1s 22ms/step - loss: 0.1826 - accuracy: 0.9604 - val_loss: 1.6114 - val_accuracy: 0.4915
Epoch 6/30
54/54 [==============================] - 1s 22ms/step - loss: 0.0881 - accuracy: 0.9837 - val_loss: 1.3511 - val_accuracy: 0.5593
Epoch 7/30
54/54 [==============================] - 1s 23ms/step - loss: 0.0798 - accuracy: 0.9767 - val_loss: 1.7381 - val_accuracy: 0.4124
Epoch 8/30
54/54 [======

In [29]:
cnn4.evaluate(X_test, y_test)

4/4 [==============================] - 0s 11ms/step - loss: 2.8426 - accuracy: 0.5690


[2.8426434993743896, 0.568965494632721]

In [30]:
cnn4.save('cnn4.h5')